# Etudes de fonctions


In [ ]:
def generate_dummy_eeg_data(self, params, buffer_duration):
        #Extract parameters from JSON dictionary
        #params = dictionnary who can include data for configure data generators
        num_channels = params.get("eeg_channels", 8)
        #eeg_channels = numbers of channels 
        #channel = represents a distinct stream of data collected from a specific electrode placed on the scalp.
        #Each electrode measures electrical activity in a specific region of the brain.
        samples_per_second = params.get("sampling_rate", 256)
        #Sampling_rate= Sampling rate per second for each channel
        epoch_period = buffer_duration
        #epoch_period prend la valeur de buffer_duration
        noise_level = params.get("noise", 1)
        #is used to recover the noise level from the parameters provided,
        #using a default value if this configuration is not specified. 
        #This allows you to adjust the amount of noise in the simulated or processed data, 
        #which is crucial for testing and evaluating the performance of algorithms under varied conditions
        artifact_prob = params.get("artifacts", 0.01)
        #artifacts represent disruptors/unwanted anomalies in EEGs
        modulation_type = params.get("modulation_type", 'sinusoidal')
        #specifies the type of modulation to apply to EEG data. Modulation types can include waveforms such as sine, square, 
        #triangular, or other shapes. 
        #The choice of modulation type can influence how data is generated or simulated.
        preset = params.get("preset", None)
        #"preset": This is the key we are looking for in the params dictionary.
        #This key is used to specify a setting or preset configuration.
        sequence = params.get("sequence", None)
        #may be used to specify a particular sequence of values or actions,This can include sequences of stimuli, events,or processing steps.
        #For example, in an experiment or simulation, you might want to define a sequence of steps or instructions to follow.
        correlation_strength = params.get("correlation_strength", 0.5)  
        # Strength of correlation between nearby channels
        power_law_slope = params.get("power_law_slope", 1.0)
        #the slope functions as an attenuation
        

        # Preset amplitude settings
        preset_settings = {
            #           del  the  alp  bet  gam
            'focus':   [0.1, 0.1, 0.5, 0.8, 0.4],
            'alert':   [0.1, 0.1, 0.4, 0.9, 0.3],
            'relaxed': [0.2, 0.2, 0.7, 0.3, 0.2],
            'drowsy':  [0.4, 0.6, 0.2, 0.2, 0.1],
        }
        #préréglages
        if preset in preset_settings:
            #This line checks if the preset variable exists in the preset_settings dictionary.
            delta_amp, theta_amp, alpha_amp, beta_amp, gamma_amp = preset_settings[preset]
            #If preset is found in preset_settings, the amplitude values for each frequency band (delta, theta, alpha, beta, gamma) 
            #are extracted and assigned to the corresponding variables.
        else:
            delta_amp = params.get("delta_amp", 0.1)
            theta_amp = params.get("theta_amp", 0.1)
            alpha_amp = params.get("alpha_amp", 0.1)
            beta_amp = params.get("beta_amp", 0.1)
            gamma_amp = params.get("gamma_amp", 0.1)
            # If preset is not found, the amplitude values are extracted from the params dictionary using the get method, 
            # which allows specifying a default value (0.1 in this case) if the key does not exist
        
        total_samples = samples_per_second * epoch_period
        # total_samples is calculated by multiplying the number of samples per second (samples_per_second) 
        # by the duration of the period (epoch_period).
        t = np.linspace(0, epoch_period, total_samples, endpoint=False)
        # t is a time vector ranging from 0 to epoch_period, containing total_samples equally spaced points. 
        # endpoint=False ensures that the upper bound is not included in the vector.
        eeg_data = np.zeros((num_channels, total_samples))
        #eeg_data is a zero-initialized array with dimensions num_channels x total_samples, 
        #thus initializing the EEG data for multiple channels (specified by num_channels) and for the total samples in the given period

        # Frequency bands
        bands = {
            'Delta': (0.5, 4),
            'Theta': (4, 8),
            'Alpha': (8, 13),
            'Beta': (13, 30),
            'Gamma': (30, 100)
        }

        amplitudes = {
            'Delta': delta_amp,
            'Theta': theta_amp,
            'Alpha': alpha_amp,
            'Beta': beta_amp,
            'Gamma': gamma_amp
        }

        # Managing the type of EEG modulation
        if modulation_type == 'sinusoidal':
            #modulation_type == 'sinusoidal': Checks if the modulation type is 'sinusoidal'.
            modulating_freq = 0.1  # frequency of the amplitude modulation
            delta_mod = (1 + np.sin(2 * np.pi * modulating_freq * t)) / 2  # between 0.5 and 1.5
            theta_mod = (1 + np.cos(2 * np.pi * modulating_freq * t)) / 2
            alpha_mod = (1 + np.sin(2 * np.pi * modulating_freq * t + np.pi / 4)) / 2
            beta_mod = (1 + np.cos(2 * np.pi * modulating_freq * t + np.pi / 4)) / 2
            gamma_mod = (1 + np.sin(2 * np.pi * modulating_freq * t + np.pi / 2)) / 2
        elif modulation_type == 'random':
            delta_mod = np.abs(np.random.randn(total_samples))
            theta_mod = np.abs(np.random.randn(total_samples))
            alpha_mod = np.abs(np.random.randn(total_samples))
            beta_mod = np.abs(np.random.randn(total_samples))
            gamma_mod = np.abs(np.random.randn(total_samples))
        
        for band, (low, high) in bands.items():
            #Iterates over each EEG frequency band defined in the bands dictionary, 
            #where band is the name of the band (e.g., 'delta') and (low, high) are the frequency limits for that band.
            amplitude = amplitudes[band]
            #Retrieves the amplitude for the current band from the amplitudes dictionary.
            freqs = np.linspace(low, high, int(samples_per_second / 2))
            #Creates a frequency vector from the low to high frequency limit for the current band. 
            #The number of points in the vector is half the sampling rate.
            power_law = freqs ** -power_law_slope
            # Applies a power law to the frequency components. The power_law_slope determines the slope of the power law, 
            # affecting the distribution of power across frequencies.

            for i in range(num_channels):
                #This loop iterates over each EEG channel. num_channels specifies the total number of EEG channels, 
                #and i is the index for the current channel.
                for f, p in zip(freqs, power_law):
                    #This inner loop iterates over each frequency (f) and its corresponding power law value (p). 
                    #The zip(freqs, power_law) pairs each frequency with its power law value, ensuring they are processed together.
                    phase = np.random.uniform(0, 2 * np.pi)
                    #A random phase is generated for the current frequency component. This phase is uniformly distributed between 0 and 
                    #2π, ensuring the signal has a random starting point in its cycle.
                    if band == 'Delta':
                        eeg_data[i] += amplitude * p * delta_mod * np.sin(2 * np.pi * f * t + phase)
                    elif band == 'Theta':
                        eeg_data[i] += amplitude * p * theta_mod * np.sin(2 * np.pi * f * t + phase)
                    elif band == 'Alpha':
                        eeg_data[i] += amplitude * p * alpha_mod * np.sin(2 * np.pi * f * t + phase)
                    elif band == 'Beta':
                        eeg_data[i] += amplitude * p * beta_mod * np.sin(2 * np.pi * f * t + phase)
                    elif band == 'Gamma':
                        eeg_data[i] += amplitude * p * gamma_mod * np.sin(2 * np.pi * f * t + phase)

        # Adding random noise
        eeg_data += noise_level * np.random.randn(num_channels, total_samples)

        # Introducing correlation between nearby channels
        for channel in range(1, num_channels):
            eeg_data[channel] += correlation_strength * eeg_data[channel - 1]

        # Introducing artifacts as random peaks
        artifact_indices = np.random.choice(total_samples, int(artifact_prob * total_samples), replace=False)
        for channel in range(0, num_channels):
            eeg_data[channel, artifact_indices] -= np.random.uniform(10, 20, len(artifact_indices))

        # Handle sequence if provided
        if sequence:
            #Vérifie si la variable sequence n'est pas vide ou nulle. Si elle contient des éléments, 
            #le code à l'intérieur du bloc sera exécuté.
            full_data = []
            #Initialise une liste vide full_data qui sera utilisée pour stocker les segments de données EEG générés.
            for seq in sequence:
                #Itère sur chaque élément dans la séquence. 
                #Chaque élément (seq) est supposé être une paire (tuple) contenant un préréglage (preset) et une durée (duration).
                preset, duration = seq
                #Déstructure le tuple en deux variables, preset et duration.
                temp_params = params.copy()
                #Crée une copie des paramètres initiaux (params) pour éviter de modifier les paramètres originaux.
                temp_params['preset'] = preset
                #Définit le préréglage dans les paramètres temporaires.
                temp_params['epoch_period'] = duration
                # Définit la durée dans les paramètres temporaires.
                temp_params['sequence'] = None
                #Assure que la séquence est définie sur None dans les paramètres temporaires 
                #pour éviter toute récursion ou réutilisation indésirable de la séquence.
                segment = generate_dummy_eeg_data(temp_params).
                #Appelle une fonction generate_dummy_eeg_data avec les paramètres temporaires pour générer un segment de données EEG synthétiques 
                #basé sur le préréglage et la durée.
                full_data.append(segment)
                #Ajoute le segment généré à la liste full_data.
            eeg_data = np.hstack(full_data)
            # np.hstack(full_data) : Concatène horizontalement tous les segments dans full_data pour former un seul ensemble de données EEG. 
            #np.hstack est une fonction NumPy utilisée pour concaténer des tableaux sur l'axe horizontal (colonnes).

        return eeg_data

#### Outer Loop (Channels): 
Iterates through each EEG channel.
#### Inner Loop (Frequencies): 
Iterates through each frequency in the band and applies the corresponding power law.
#### Random Phase: 
Generates a random phase for each frequency component to ensure the resulting signal is not phase-locked.
#### Amplitude Modulation: 
Applies the band-specific amplitude modulation (delta_mod, theta_mod, etc.).
#### Summation: 
Adds the modulated sine wave to the EEG data for the current channel.